In [ ]:
from transformers import pipeline
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

#print(torch.cuda.device_count())
#os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

#モデル読み込み
model_name = "meta-llama/Llama-2-7b-chat-hf"

model_name = "tokyotech-llm/Swallow-MS-7b-v0.1"
model_name = "cyberagent/open-calm-7b"

model_name="Local-Novel-LLM-project/Vecteus-v1"
model_name="hatakeyama-llm-team/tanuki_inst_0515test"
model_name="kanhatakeyama/0516tanuki_lr5e5_epoch1"
#model_name="../X_merge/merged_models/0515_1_3_6"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)


In [ ]:

tokens=[5092, 272, 1045, 2850, 327, 8,11267, 269, 564, 298, 2]
tokenizer.decode(tokens,)

In [ ]:
t_list=[
"\n\n### 応答:\n",
"\n\n### 応答:\n「",
"\n\n### 応答:\nああ",
"\n\n### 応答:\naaa",
"1\n\n### 応答:\naaa",
"あ\n\n### 応答:\naaa",
"」\n\n### 応答:\naaa",
]
t_list=[
    "",
    "a \n a",
"a\n   ",
"\n\n### 指示:\n",
"\n\n### 指示:\n「",
"\n\n### 指示:\nああ",
"\n\n### 指示:\naaa",
"1\n\n### 指示:\naaa",
"あ\n\n### 指示:\naaa",
"」\n\n### 指示:\naaa",
]
for t in t_list:
    enc=tokenizer.encode(t)
    print(enc,tokenizer.decode(enc))

tokenizer.decode(enc)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                            #quantization_config=bnb_config, 
                                            device_map="auto",
                                            torch_dtype=torch.bfloat16,
                                            )


In [ ]:
#model.load_adapter(model_path)

In [ ]:
#層の表示
for name, param in model.named_parameters():
    print(name)
    if "gate" in name:
        print("***",name)

In [ ]:
def perplexity(model, tokenizer, text) -> torch.Tensor:
    tokenized_input = tokenizer.encode(
        text, add_special_tokens=False, return_tensors="pt"
    ).to(model.device)
    with torch.inference_mode():
        output = model(tokenized_input, labels=tokenized_input)
    ppl = torch.exp(output.loss)
    return ppl.item()



In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=50, 
              repetition_penalty=1.2,
              temperature=0.6,
              #repetition_penalty=1.,
              #temperature=0.1,
              #top_p=1.0,
              #top_k=0.
              )


In [ ]:
question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n"
answer_template="\n\n### 応答:\n"
answer_template="\n\n### 応答:\n"

def gen_prompt(q):
    return f"{question_template}{q}{answer_template}"
    return question_template+q+answer_template

gen_prompt("こんにちは")

In [ ]:
questions=[
"こんにちは",
"元気ですか?",
"いい天気ですね",
"明日の天気は?",
"計算してください 1+1はいくつですか｡",
"フィボナッチ数列を生成するpythonのコードを作成してください",
"ドラえもんの友達はだれですか?",
"レイリー散乱とはなんですか",
"hello!",
"今の天気は晴れです｡この結果をjsonで出力してください｡ ",
"純粋理性批判はたぬきに理解できますか?",
]
questions=[
"たぬきに純粋理性批判は理解できますか?",
"フィボナッチ数列を生成するpythonのcode",
"元気ですか?",
 "日本の首相は?",
"東京科学大学の学長は?",
 "東京科学大学とは?"   ,
]
#questions=[
#    "アスキーアートを出力して",
#]

for question in questions:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    res=pipe(inp,max_new_tokens=256)[0]["generated_text"][len(inp):]
    print(res)
    #print(tokenizer.encode(res)[:4])

In [ ]:
#tokenizer.piece_size = 100000
#tokenizer.add_special_tokens({'additional_special_tokens': ['\\n']})
#tokenizer.add_prefix_space=False
inp="フィボナッチ数列を生成するpythonのコード"

inp=gen_prompt("フィボナッチ数列を生成するpythonのコードを作成してください")
res=pipe(inp)[0]["generated_text"]
res_cut=res[len(inp):]
#tokenizer.encode("### 応答:\n はい、可能です")
print(inp)
print(res_cut)
tokenizer.encode(res)[:4]

In [ ]:
res
tokenizer.encode(res)[:4]
res

# ベンチマーク問題での応答評価

In [ ]:
pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=300, 
              repetition_penalty=1.,
              temperature=0.1,
              top_p=1.0,
              top_k=0.
              )

In [ ]:
#ベンチマーク問題への応答性の軽い評価
import pandas as pd
jaster_path="bench_data/jaster.txt"
with open(jaster_path,"r") as f:
    input_list=f.readlines()

for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:
jmt_path="bench_data/jmt.csv"
df=pd.read_csv(jmt_path)
input_list=df["input"].to_list()
for question in input_list:
    #question_template="以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。<SEP>指示<SEP>"
    #inp=f"{question_template}{question}<SEP>応答<SEP>"
    inp=gen_prompt(question)
    #inp=f"質問: {question}回答: "
    print("--------------------")
    print(question)
    print("出力")
    print(pipe(inp)[0]["generated_text"][len(inp):])

In [ ]:

pipe=pipeline('text-generation',model=model,tokenizer=tokenizer, 
              max_new_tokens=300, 
              #repetition_penalty=1.,
              temperature=0.1,
              #top_p=1.0,
              #top_k=0.,
              do_sample=True,
              )

question="""日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。"""
inp=gen_prompt(question)
r=pipe(inp)[0]["generated_text"][len(inp):]
print(r)

In [ ]:
prompt=inp
do_sample=True
#do_sample=False
temperature=0.1
max_new_token=30

question="""日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。"""
#question='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:\n'
#question='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:'
inp=gen_prompt(question)

inp='以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。\n\n### 指示:\n日本の独占禁止法と市場競争への影響について説明してください。具体的な事例を挙げて比較・検討してみましょう。\n\n### 応答:'
def ask(prompt):
    input_ids = tokenizer([prompt]).input_ids
    output_ids = model.generate(
        torch.as_tensor(input_ids).cuda(),
        do_sample=do_sample,
        temperature=temperature,
        max_new_tokens=max_new_token,
        #repetition_penalty=1.0,
    )


    output_ids_=output_ids[0][len(input_ids[0]) :]
    output_text = tokenizer.decode(output_ids_, skip_special_tokens=True)
    return output_text

for i in range(3):
    print(i)
    print(ask(inp))